In [1]:
import boto3
import pandas as pd

s3_client = boto3.client('s3')

In [2]:
bucket = 'hbomax-datascience-development-dev'

In [3]:
# collect column headers for the SHAP columns
shap_cols_df = pd.read_csv(f's3://{bucket}/free_trial_model/snowflake-hbomax-staging/schema/column_headers.csv',header=None)
shap_column_headers = list(shap_cols_df[shap_cols_df.columns[0]])

# include additional columns output by the batch transform job
shap_column_headers = ['PREDICTION','SHAP_EXPECTED'] + shap_column_headers

In [4]:
# collect output files from the batch transform
response = s3_client.list_objects_v2(
        Bucket=bucket,
        Prefix ='free_trial_model/snowflake-hbomax-staging/results')
    
files = [r['Key'] for r in response['Contents'] if r['Size'] > 0]

raw_data = [ pd.read_csv(
        f's3://{bucket}/{file}', header=None) for file in files ]

concat_data = pd.concat(raw_data)

In [5]:
# define the columns we want to keep from the original output 
keep_cols = concat_data.columns[-len(shap_column_headers):]

# add PERIOD_RANK columns
period_rank_index = shap_column_headers.index('PERIOD_RANK') - 1
keep_cols = keep_cols.insert(loc=0, item=period_rank_index)

# add HBO_UUID columns
keep_cols = keep_cols.insert(loc=0, item=0)

In [6]:
# create a DataFrame with the formatted output
shap_df = concat_data[keep_cols]

In [7]:
# rename column headers
shap_df.columns = ['HBO_UUID','PERIOD_RANK'] + shap_column_headers

In [8]:
# formatted output DataFrame 
shap_df.head()

,HBO_UUID,PERIOD_RANK,PREDICTION,SHAP_EXPECTED,one_hot__x0_apple,one_hot__x0_dtc,one_hot__x0_google,one_hot__x0_samsung,one_hot__x1_missing,one_hot__x1_movies,...,HBOMAX_ORIGINAL_CONTENT_STREAMING_TIME_PERC_ADJ,HBOMAX_ORIGINAL_CONTENT_NUM_EPI_COMPLETED_80_PERC_ADJ,HBO_ACQUIRED_CONTENT_STREAMING_TIME_PERC_ADJ,HBO_ACQUIRED_CONTENT_NUM_EPI_COMPLETED_80_PERC_ADJ,HBOMAX_ACQUIRED_CONTENT_STREAMING_TIME_PERC_ADJ,HBOMAX_ACQUIRED_CONTENT_NUM_EPI_COMPLETED_80_PERC_ADJ,FRIENDS_STREAMING_SEC_ADJ,FRIENDS_STREAMING_PERC_ADJ,BBT_STREAMING_SEC_ADJ,BBT_STREAMING_PERC_ADJ
0,edc75618b486e34a94642eaa2112aca8,1.0,0.349200,0.151332,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-0.001139,0.0,0.0,0.0,0.0,-0.002234,0.0,0.0
1,256f2af427159566e04c47424a5ee4a3,2.0,0.441185,0.151332,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.020160,0.0,0.0,0.0,0.0,-0.001108,0.0,0.0
2,b34c7ac3994072ceca5577d34c57fe7e,1.0,0.491843,0.151332,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.001846,0.0,0.0,0.0,0.0,-0.003097,0.0,0.0
3,81ad19e10c7ff9abe94a52ebe33acdab,4.0,0.681006,0.151332,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-0.000959,0.0,0.0,0.0,0.0,-0.001058,0.0,0.0
4,780df7152fcc3df8eebe966003f7df34,6.0,0.681006,0.151332,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.002636,0.0,0.0,0.0,0.0,-0.001058,0.0,0.0


In [12]:
shap_df['PERIOD_RANK']

,PERIOD_RANK,PERIOD_RANK
0,1.0,0.003874
1,2.0,0.009922
2,1.0,0.001273
3,4.0,-0.001728
4,6.0,-0.001147
...,...,...
95,3.0,-0.001916
96,3.0,-0.001831
97,5.0,-0.037073
98,2.0,0.001941
